# Device Communication Basics

## Overview
This document covers the basics of device communication and configuration. OpenSync devices employ USB on-the-go (OTG), so no drivers are necessary to communicate with an OpenSync device. This is beneficial as device discovery revolves arounds a computer's ability to detect a USB device that is actively connected to the computer.

## Purpose
This example demonstrates how to search, connect, and "talk" to an OpenSync device. OpenSync relies on a stable USB connection, so make sure one is present for this demonstration.

## Imports
To begin, import opensync as such.

In [1]:
from opensync import opensync

## Device Connection
We need to search for any OpenSync devices to connect to. This is performed by the function `device_comm_search`.

In [2]:
avail_ports = opensync.device_comm_search()
avail_ports

['COM7']

There are two ways we can open a connection once we found the correct port to connect to. We can manually manage the resources with `device_comm_open` and `device_comm_close` or use context managers to automatically manage resources with `device_comm_managed`. Both methods will be shown in this document. 

### Manually Open Device Connection


In [3]:
# Open USB port
sync_device = opensync.device_comm_open(avail_ports[0])
sync_device

Serial<id=0x26e1a6390f0, open=True>(port='COM7', baudrate=19200, bytesize=8, parity='N', stopbits=1, timeout=0.1, xonxoff=False, rtscts=False, dsrdtr=False)

In [4]:
# Get device state and version
print(opensync.device_system_version(sync_device))
print(opensync.device_system_status(sync_device))

['OpenPIV', 'OpenSync', '5A3F6CE6128EEB36', '0.2.0']
['IDLE']


In [5]:
# Close USB port
opensync.device_comm_close(sync_device)

### Open with Context Manager

In [6]:
with opensync.device_comm_managed(avail_ports[0]) as sync_device:
    # Get device state and version
    print(opensync.device_system_version(sync_device))
    print(opensync.device_system_status(sync_device))

['OpenPIV', 'OpenSync', '5A3F6CE6128EEB36', '0.2.0']
['IDLE']


## Status Information
OpenSync allows one to poll system status information even when running a pulse sequence. Below are the most commonly used requests.

In [7]:
with opensync.device_comm_managed(avail_ports[0]) as sync_device:
    print(opensync.device_system_version(sync_device))
    print(opensync.device_system_status(sync_device))
    print(opensync.device_system_freq(sync_device))

['OpenPIV', 'OpenSync', '5A3F6CE6128EEB36', '0.2.0']
['IDLE']
['pll_sys = 250000kHz', 'pll_usb = 48000kHz', 'rosc = 10193kHz', 'clk_sys = 250000kHz', 'clk_peri = 48000kHz', 'clk_usb = 48000kHz', 'clk_adc = 48000kHz']


## Executing Device Parameters
Two of the more important functions for OpenSync are `device_system_fire` and `device_system_stop`. `device_system_fire` executes a pulse sequence with the current pulse and clock configrations. To prematurely stop a pulse sequence iteration, one can use `device_system_stop` and an abort signal will be sent to the OpenSync device. OpenSync allows for one to repeat a pulse sequence up to 500,000 times allowing for up to 500,000 image pairs to be collected during an experiment.

It should be noted that the below example sets an internal debug setting to level 1. Internal debug logs may be useful when trying to figure out why some unexpected thing happened. It should be noted that querying the device during operation is allowed, unlike setting debug levels and running a sequence which can only occur during device standby/idle. Setting debug levels should be a last resort when troubleshooting your device. A quick reference for debug levels is given below.
 - `0`: Do not show internal debugging logs
 - `1`: Print most internal debugging logs, but abort pulse sequence
 - `2`: Print all internal debugging logs without aborting

In [8]:
with opensync.device_comm_managed(avail_ports[0]) as sync_device:
    # Set debug status to 1 (prints internal logs then aborts sequence)
    opensync.device_comm_write(
        sync_device,
        ':device:debug 1'
    )

    # Execute with current pulse and clock configs
    resp = opensync.device_system_fire(sync_device)

    for response in resp:
        print(response)
    
    # Emits error because debug level 1 aborts the sequence
    print(opensync.device_system_stop(sync_device))

    # Print device status after debug
    print(opensync.device_system_status(sync_device))

Clock config id: 0
Clock config is active: 0
Clock config clock type: 0
Clock config clock divider: 1
Clock config sm: 0
Clock config dma channel: 0
Clock config clock pin: 16
Clock config trigger pin: 13
Clock instruction 0: reps=0; delay=0
Clock instruction 1: reps=0; delay=0
Clock instruction 2: reps=0; delay=0
Clock instruction 3: reps=0; delay=0
Clock instruction 4: reps=0; delay=0
Clock instruction 5: reps=0; delay=0
Clock instruction 6: reps=0; delay=0
Clock instruction 7: reps=0; delay=0
Trigger instruction 0: skips=0; delay=0
Trigger instruction reps: 0
Clock config id: 1
Clock config is active: 0
Clock config clock type: 0
Clock config clock divider: 1
Clock config sm: 1
Clock config dma channel: 0
Clock config clock pin: 17
Clock config trigger pin: 13
Clock instruction 0: reps=0; delay=0
Clock instruction 1: reps=0; delay=0
Clock instruction 2: reps=0; delay=0
Clock instruction 3: reps=0; delay=0
Clock instruction 4: reps=0; delay=0
Clock instruction 5: reps=0; delay=0
Cloc